In [1]:
from Loader import Loader

loader = Loader()
files = loader.get_all_levels()
txt_level = loader.get_txt_level(files[3])

level, state = loader.load_level(txt_level)

In [2]:
level.get_board()

((0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 1, 0, 1, 0),
 (0, 1, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 1, 1, 0),
 (0, 1, 1, 1, 1, 1, 0),
 (0, 0, 0, 0, 0, 0, 0))

In [3]:
# Cargar Celda

def is_valid(board, coord):
    try:
        x, y = coord
        return board[x][y] == 1
    except IndexError:
        return False

def is_rocks_free(board, player, rocks, coord):
    return (is_valid(board, coord)
            and not tuple(coord) in rocks)

def is_cross_free(board, player, crosses, coord):
    return (is_valid(board, coord) and not tuple(coord) in crosses)

def is_player_free(board, player, rocks, water, crosses, coord):
    return (is_rocks_free(board, player, rocks, coord) and is_cross_free(board, player, crosses, coord) and not tuple(coord) in water)

In [4]:
# Cargar Celda

def remove_crosses_from(crosses, enemy, coord):
    new_crosses = set(crosses)
    sum_initial_cross = sum(coord)
    enemy_is_facing_down = sum(enemy) < sum_initial_cross

    for cross in crosses:
        cross_sum = sum(cross)
        if ((enemy_is_facing_down
             and cross_sum >= sum_initial_cross)
            or
            (not enemy_is_facing_down
                and cross_sum <= sum_initial_cross)):
            new_crosses.remove(cross)

    return new_crosses

In [5]:
# Cargar Celda

import numpy as np
from JuegoEscapa import State

def move_player(board, player, enemy, rocks, water, crosses, keys, target, mov, player_has_key):
    next_coord = tuple(np.add(player, mov))

    if is_player_free(board, player, rocks, water, crosses, next_coord):
        if next_coord == target:
            # If players wins => Remove player.
            if player_has_key:
                player = None
        else:                
            player = list(next_coord)
            # If players picks the key => Remove key.
            if next_coord in keys:
                keys.remove(next_coord)
                player_has_key = True

    return State(player, rocks, water, crosses, keys, player_has_key)


def move_rocks(enemy, board, player, rocks, water, crosses, keys, target, mov, player_has_key):
    next_coord = tuple(np.add(player, mov))
    next_rock_space = tuple(np.add(next_coord, mov))

    if (next_coord in rocks
            and is_rocks_free(board, player, rocks, next_rock_space) and next_rock_space != target):
        rocks.remove(next_coord)
        # We check if water in new rock position.
        if next_rock_space in water:
            water.remove(next_rock_space)
        else:
            if next_rock_space in crosses:
                crosses = remove_crosses_from(crosses, enemy, next_rock_space)
            rocks.add(next_rock_space)

    return State(player, rocks, water, crosses, keys, player_has_key)

def move(level, state, mov):
    pos_movements = ((1, 0), (-1, 0), (0, 1), (0, -1))

    board = level.get_board()
    target = level.get_target()
    enemy = level.get_enemy()

    player = state.get_player()
    rocks = state.get_rocks()
    water = state.get_water()
    crosses = state.get_crosses()
    keys = state.get_keys()
    player_has_key = state.get_player_has_key()

    if tuple(mov) in pos_movements and player:
        s1 = move_rocks(
            enemy, board, player, rocks, water, crosses, keys, target, mov, player_has_key)
        s2 = move_player(
            board, player, enemy, s1.get_rocks(), s1.get_water(), s1.get_crosses(), keys, target, mov, player_has_key)
        return s2

    return state

In [6]:
# Cargar Celda

def is_goal(state):
    player = state.get_player()
    return not player

In [7]:
from ipywidgets import HTML
from Gui import Gui

ui = Gui()
s1 = state

html_str = ui.get_html(level, state)
HTML(value = html_str)

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [8]:
from IPython.display import display
from MediadorVPedro import Mediator
from Gui import Gui

import warnings
warnings.filterwarnings("ignore")
ui = Gui(manual = True)

med = Mediator.get_instance(modelPath = "JuegoEscapa.ipynb")   
med.register_ui(ui)


# Cuando llamo a la función se crean
display(ui.get_ui_elements())

### Varios enemigos y que las cuces no se quiten.